In [1]:
import pandas as pd
import sys
sys.path.insert(0, '/home/istywhyerlina/PipelinePyspark/exercise4/data_pipeline_exercise_4/script')
from helper.db_conn import db_connection
from pangres import upsert
from etl.extract import *
from etl.load import *

from sqlalchemy import *


In [2]:
df1=extract_sheet(key_file, "brand_car")
load_stg(df1, "car_brand")
df2=extract_db("car_sales", "src")
df2=df2.drop(['model','trim','body','vin','seller','saledate'], axis=1)
load_stg(df2, "car_sales")
df3=extract_api("https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/us_states_data/refs/heads/main/us_states.json")
load_stg(df3, "us_state")


Extract Data Brand Car Spreadsheet from Source is Succeed
Load data to car_brand is succeed
Extract Data car_sales from Source is Succeed
Load data to car_sales is succeed
Extract Data us_state from API source is Succeed
Load data to us_state is succeed


In [3]:
_, stg_engine, dwh_engine,_ = db_connection()

In [4]:
conn=stg_engine.connect()
meta = MetaData()
meta.reflect(bind=stg_engine)


In [5]:
for table in meta.sorted_tables:
    globals()[f'{table}']=extract_db(str(table), "stg")
    

Extract Data car_brand from Staging is Succeed
Extract Data car_sales from Staging is Succeed
Extract Data us_state from Staging is Succeed


In [6]:

car_brand= car_brand.drop('created_at', axis=1)
car_brand = car_brand.rename({'brand_car_id': 'brand_car_id_nk'}, axis='columns')


In [7]:
get_key={
            "car_brand":"brand_car_id_nk",
            "car_sales":"id_sales_nk",
            "us_state":"id_state_nk"
        }
car_brand=car_brand.set_index([get_key.get('car_brand')], drop = False)
car_brand= car_brand.drop('brand_car_id_nk', axis=1)


In [8]:
us_state= us_state.drop('created_at', axis=1)
us_state = us_state.rename({'id_state': 'id_state_nk'}, axis='columns')

In [9]:
get_key={
            "car_brand":"brand_car_id_nk",
            "car_sales":"id_sales_nk",
            "us_state":"id_state_nk"
        }
us_state=us_state.set_index([get_key.get('us_state')], drop = False)
us_state= us_state.drop('id_state_nk', axis=1)

In [10]:
car_brand_dwh=extract_db(str('car_brand'), "dwh")
car_brand_dwh=car_brand_dwh[['brand_car_id','brand_name']]

Extract Data car_brand from Datawarehouse is Succeed


In [11]:
us_state_dwh=extract_db(str('us_state'), "dwh")
us_state_dwh=us_state_dwh[['id_state','code']]


Extract Data us_state from Datawarehouse is Succeed


In [12]:
car_sales=car_sales.merge(car_brand_dwh, how="left", left_on='brand_car', right_on='brand_name')
car_sales=car_sales.merge(us_state_dwh, how="left", left_on='state', right_on='code')


In [13]:
car_sales=car_sales.drop(['brand_car','state','brand_name','code','created_at'], axis=1)

In [14]:
car_sales

,id_sales,year,transmission,condition,odometer,color,interior,mmr,sellingprice,brand_car_id,id_state
0,46,2013,automatic,48.0,29578.0,black,black,19500.0,22200.0,c06cc666-8b4e-4daa-b3f1-6cc43e405cdb,d7f844f9-06ba-4804-82b0-5d61faf555bd
1,47,2012,automatic,46.0,49014.0,—,gray,32000.0,32250.0,6856870a-095d-46a3-aa42-12273a8a14f2,1a7e914b-4b00-4553-aec9-e9a494342ce4
2,48,1998,automatic,21.0,287017.0,white,beige,700.0,1150.0,39a98c84-05db-4bb0-9e4d-b17646199e6f,1a7e914b-4b00-4553-aec9-e9a494342ce4
3,49,2004,automatic,26.0,169982.0,gray,gray,4375.0,4600.0,edac340a-954d-4075-a48f-53076f519623,b3a42363-6957-48f2-adbf-093c97211149
4,50,2012,automatic,34.0,34930.0,blue,gray,19550.0,18200.0,39a98c84-05db-4bb0-9e4d-b17646199e6f,55657107-f160-419a-8e00-caa8db24a294
...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,2012,automatic,36.0,103016.0,white,gray,17700.0,16800.0,d7a04250-036f-421e-b0bb-ea915e217978,1c4a6c8f-6e07-4ce2-91d6-e48ff84ded94
29996,29997,2012,automatic,36.0,41092.0,—,tan,8475.0,9800.0,c06cc666-8b4e-4daa-b3f1-6cc43e405cdb,f8cd520b-4af5-4c4c-a876-7a8625dda988
29997,29998,2003,,19.0,292925.0,white,gray,3225.0,2400.0,c2ca5c7c-e1dd-4e63-abbe-d96d404a6693,1ec45f88-fd53-4ad4-9194-44954fe925b1
29998,29999,2014,automatic,4.0,25083.0,gray,gray,12900.0,12800.0,edac340a-954d-4075-a48f-53076f519623,f8cd520b-4af5-4c4c-a876-7a8625dda988


In [15]:
car_sales = car_sales.rename({'id_sales': 'id_sales_nk', 'sellingprice':'selling_price'}, axis='columns')

In [16]:
get_key={
            "car_brand":"brand_car_id_nk",
            "car_sales":"id_sales_nk",
            "us_state":"id_state_nk"
        }
car_sales=car_sales.set_index([get_key.get('car_sales')], drop = False)
car_sales= car_sales.drop('id_sales_nk', axis=1)

In [18]:
to_load=['us_state','car_brand','car_sales']
for table in to_load:
    load_dwh(globals()[f'{table}'],table)

Load data us_state to Datawarehouse  is succeed
Load data car_brand to Datawarehouse  is succeed
Load data car_sales to Datawarehouse  is succeed


In [196]:
upsert(con=dwh_engine, 
        df=us_state, 
        table_name="us_state", 
        schema="public", 
        if_row_exists="update")

In [197]:
upsert(con=dwh_engine, 
        df=car_brand, 
        table_name="car_brand", 
        schema="public", 
        if_row_exists="update")

In [198]:
upsert(con=dwh_engine, 
        df=car_sales, 
        table_name="car_sales", 
        schema="public", 
        if_row_exists="update")

In [ ]:
import requests

In [ ]:
# Establish connection to API
resp = requests.get("https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/us_states_data/refs/heads/main/us_states.json")

# Parse the response JSON
raw_response = resp.json()

# Convert the JSON data to a pandas DataFrame
df_api = pd.DataFrame(raw_response['regions'])


In [ ]:
print(df_api)

    id_state code                      name
0          1   al                   Alabama
1          2   ak                    Alaska
2          3   az                   Arizona
3          4   ar                  Arkansas
4          5   ca                California
..       ...  ...                       ...
63        64   pr               Puerto Rico
64        65   vi       U.S. Virgin Islands
65        66   gu                      Guam
66        67   mp  Northern Mariana Islands
67        68   as            American Samoa

[68 rows x 3 columns]
